# Simple synchronous parallel evaluation

This demonstrates a very simple synchronous evaluation model, which is essentially a map/reduce approach to concurrently evaluating individuals.  This lends itself naturally to a by-generation implementation, which we show here.

In [1]:
import sys
from dask.distributed import Client, LocalCluster
import toolz

from leap import core
from leap import ops
from leap import binary_problems
from leap import util
from leap.parallel import sync

First, let's set up `dask` to run on local pretend"cluster" on your machine.

In [2]:
cluster = LocalCluster()
client = Client(cluster)

Now create an initial random population of five individuals that use a binary representation for solving the MAX ONES problem.

In [11]:
parents = core.Individual.create_population(5,
                                            initialize=core.create_binary_sequence(4),
                                            decoder=core.IdentityDecoder(), problem=binary_problems.MaxOnes())

However, we need to evaluate this initial population *before* we fall into the by-generation loop since we select parents based on their fitness, and that implies that they've already been evaluated.  But, since we want to use a sychronous, concurrent evaluation mechanism throughout this exsmple, we will use `sync.eval_population()`

In [12]:
parents = sync.eval_population(parents, client=client)

Now show that the initial population has, indeed, been evaluated by all the dask workers.

In [13]:
[print(x.genome, x.fitness) for x in parents]

[0, 0, 1, 0] 1
[0, 0, 1, 1] 2
[0, 1, 0, 1] 2
[1, 1, 0, 1] 3
[0, 1, 1, 1] 3


[None, None, None, None, None]

Now we're ready to run for five generations.  For each generation we'll again concurrently evaluate all the offspring before proceeding to the next generation.

In [14]:
for current_generation in range(5):
    offspring = toolz.pipe(parents,
                         ops.tournament,
                         ops.clone,
                         ops.mutate_bitflip,
                         ops.uniform_crossover,
                         sync.eval_pool(client=client, size=len(parents)))
    
    print('generation:', current_generation)
    [print(x.genome, x.fitness) for x in offspring]
    
    parents = offspring

generation: 0
[1, 1, 0, 1] 3
[0, 1, 1, 1] 3
[0, 0, 1, 1] 2
[1, 0, 0, 0] 1
[0, 1, 1, 1] 3
generation: 1
[1, 1, 1, 0] 3
[1, 1, 0, 1] 3
[0, 1, 0, 1] 2
[1, 1, 1, 1] 4
[0, 1, 0, 1] 2
generation: 2
[0, 0, 1, 1] 2
[1, 1, 1, 0] 3
[0, 1, 0, 1] 2
[1, 1, 0, 1] 3
[0, 1, 0, 1] 2
generation: 3
[0, 1, 0, 1] 2
[1, 0, 1, 0] 2
[1, 1, 1, 1] 4
[0, 1, 0, 0] 1
[0, 1, 0, 0] 1
generation: 4
[0, 0, 1, 1] 2
[1, 0, 0, 1] 2
[1, 0, 0, 1] 2
[1, 1, 0, 1] 3
[1, 0, 0, 1] 2


In [10]:
[print(x.genome, x.fitness) for x in parents]

[0, 1, 0, 0] 1
[0, 1, 1, 1] 3
[0, 0, 0, 0] 0
[0, 0, 1, 1] 2
[1, 1, 0, 1] 3


[None, None, None, None, None]